In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
import pandas as pd
from IPython.core.display import display, HTML
# %matplotlib notebook
# %matplotlib notebook
import matplotlib.pyplot as plt
# %matplotlib notebook
# %matplotlib notebook
import warnings
warnings.filterwarnings('ignore')

%reload_ext autoreload
%autoreload 1
%matplotlib inline

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

import os, glob
import cv2
import dlib
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, euclidean
from scipy import stats
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy

# import the necessary packages
from skimage import feature, io, color
from scipy.stats import moment, describe
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import statsmodels.formula.api as smfapi
import statsmodels.api as smapi
from scipy.stats import f as f_test

from Utils.DataUtils import *
from Utils.UtilsViz import *


import warnings
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE

In [8]:
train_features = pd.read_csv("./data/train_values.csv")
train_labels = pd.read_csv("./data/train_labels.csv")
test_features = pd.read_csv("./data/test_values.csv")

In [13]:
display(
    train_features.head(),
    train_features.shape)

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,0z64un,1,normal,128,2,0,0,2,308,0.0,1,45,170,0
1,ryoo3j,2,normal,110,3,0,0,0,214,1.6,0,54,158,0
2,yt1s1x,1,normal,125,4,3,0,2,304,0.0,1,77,162,1
3,l2xjde,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0
4,oyt4ek,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0


(180, 14)

In [27]:
analyse_nans(train_features)

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
total,0,0,0,0,0,0,0,0,0,0,0,0,0,0
percentage,0,0,0,0,0,0,0,0,0,0,0,0,0,0
idx_list,[],[],[],[],[],[],[],[],[],[],[],[],[],[]


In [24]:
cat_vars = ["slope_of_peak_exercise_st_segment", "thal", "chest_pain_type", "resting_ekg_results", "sex", "exercise_induced_angina"]
for cv in cat_vars:
    train_features[cv] = train_features[cv].astype("category")
train_features.head()

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,0z64un,1,normal,128,2,0,0,2,308,0.0,1,45,170,0
1,ryoo3j,2,normal,110,3,0,0,0,214,1.6,0,54,158,0
2,yt1s1x,1,normal,125,4,3,0,2,304,0.0,1,77,162,1
3,l2xjde,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0
4,oyt4ek,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0


In [30]:
dummy_df = pd.get_dummies(train_features[cat_vars], drop_first=True)
train_features[[c for c in train_features.columns if c not in cat_vars]]

,age,chest_pain_type_2,chest_pain_type_3,chest_pain_type_4,exercise_induced_angina_1,fasting_blood_sugar_gt_120_mg_per_dl,max_heart_rate_achieved,num_major_vessels,oldpeak_eq_st_depression,patient_id,resting_blood_pressure,resting_ekg_results_1,resting_ekg_results_2,serum_cholesterol_mg_per_dl,sex_1,slope_of_peak_exercise_st_segment_2,slope_of_peak_exercise_st_segment_3,thal_normal,thal_reversible_defect
0,45.0,NaN,NaN,NaN,NaN,0.0,170.0,0.0,0.0,0z64un,128.0,NaN,NaN,308.0,NaN,NaN,NaN,NaN,NaN
1,54.0,NaN,NaN,NaN,NaN,0.0,158.0,0.0,1.6,ryoo3j,110.0,NaN,NaN,214.0,NaN,NaN,NaN,NaN,NaN
2,77.0,NaN,NaN,NaN,NaN,0.0,162.0,3.0,0.0,yt1s1x,125.0,NaN,NaN,304.0,NaN,NaN,NaN,NaN,NaN
3,40.0,NaN,NaN,NaN,NaN,0.0,181.0,0.0,0.0,l2xjde,152.0,NaN,NaN,223.0,NaN,NaN,NaN,NaN,NaN
4,59.0,NaN,NaN,NaN,NaN,0.0,145.0,0.0,4.2,oyt4ek,178.0,NaN,NaN,270.0,NaN,NaN,NaN,NaN,NaN
5,42.0,NaN,NaN,NaN,NaN,0.0,150.0,0.0,0.0,ldukkw,130.0,NaN,NaN,180.0,NaN,NaN,NaN,NaN,NaN
6,60.0,NaN,NaN,NaN,NaN,0.0,157.0,2.0,2.6,2gbyh9,150.0,NaN,NaN,258.0,NaN,NaN,NaN,NaN,NaN
7,57.0,NaN,NaN,NaN,NaN,0.0,112.0,1.0,0.6,daa9kp,150.0,NaN,NaN,276.0,NaN,NaN,NaN,NaN,NaN
8,59.0,NaN,NaN,NaN,NaN,0.0,140.0,0.0,3.4,3nwy2n,170.0,NaN,NaN,326.0,NaN,NaN,NaN,NaN,NaN
9,50.0,NaN,NaN,NaN,NaN,0.0,158.0,0.0,1.6,1r508r,120.0,NaN,NaN,219.0,NaN,NaN,NaN,NaN,NaN
